# 3D Image Analysis Example
This purpose of this Jupyter notebook is to show how to perform some basic image analysis task on the Aurora cluster at Lunarc. The example data is acquired at the 4D Imaging Lab at Lund University. 
## Step 0: Getting the code
Clone the QIM@LU GitHub repository by running 
```bash
git clone https://github.com/jhektor/qim.git
```
in a terminal on Aurora. This will create a folder called qim which contains the code.

## Step 1: Setting up the python environment
Before running this code you'll need to create a python environment for it. This is done by running the following commands in a terminal on Aurora:
```bash
module load anaconda3
conda env create -f path_to_qim/qim/envs/qim_env.yml
source activate qim
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install jupyter-matplotlib
jupyter nbextension enable --py widgetsnbextension
```

This sets up a python3 environment containing [Jupyterlab](https://jupyterlab.readthedocs.io/en/latest/getting_started/installation.html#) and [dxchange](https://dxchange.readthedocs.io/en/latest/) as well as some standard python libraries such as numpy and matplotlib. You will only need to do this step the first time you want to run the code, for further use the following commands are enough
```bash
module load anaconda3
source activate qim
```
Once the python environment is set you can open this notebook in your browser by running `jupyter lab foam_ex.ipnb` in the same terminal.

## Step 2: Importing python modules
The code below will import the python modules we will need for this example. If the dxchange module is not available the reading of the example data will be slower but it should still work.

In [1]:
try:
    import dxchange.reader as dxreader
    DXCHANGE = True
except ImportError:
    print('dxchange not available, reading data will be slower')
    print('This is not yet implemented. Please install dxchange')
    DXCHANGE = False
    exit
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib
from scipy import ndimage as ndi 
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, h_maxima
from skimage import measure


## Step 3: Reading example data
The example data is a tomography scan of a cellulose based foam. More details can be found in this [paper](https://www.sciencedirect.com/science/article/pii/S0021979716301734?via%3Dihub). The data set consist of 400 tif images, each one represents a slice through the sample. The code below reads the dataset using dxchange and plots the first slice. It also creates a mask of the region containing the sample

In [2]:
filepath = '/home/hektor/data/foam/foam2_40kV_3micron_16bit/foam2_40kV_3micron0000.tif'
if DXCHANGE:
    ind = np.arange(0,400)
    data = dxreader.read_tiff_stack(filepath,ind)
plotslice = 200
plt.figure()
plt.imshow(data[plotslice],plt.cm.gray)
plt.suptitle('Center slice')
#Create mask
mask = np.where(data>1,True,False)
plt.figure()
plt.suptitle('Mask')
plt.imshow(mask[plotslice])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Step 4: Thresholding

The goal of the thresholding is to identify the different phases present in the sample. In our case this means to separate the cell walls of the foams from the air inside the cells.
For thresholding it can be very useful to look at the histogram of all images. To better see smaller peaks we plot the histogram in logarithmic scale.

In [3]:
hist,be = np.histogram(data[mask],bins=1000)
bc = (be[1:] + be[:-1])/2 #bin centers (arithmetic mean)
plt.figure()
plt.semilogy(bc,hist)
print('Intensity of the main peak is ', bc[np.argmax(hist)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Intensity of the main peak is  9274.9195


The intensity of the main peak is roughly 9300. As a first test let's see what happens if we make a binary image where 0 and 1 represents intensities above and below than 10000, respectively.

In [4]:
threshold = 10000
binary = np.where(data>threshold,0,1)
fig,axes = plt.subplots(1,3)
ax = axes.ravel()
ax[0].imshow(binary[plotslice],cmap=plt.cm.gray)
ax[0].set_title('Top view')
ax[1].imshow(binary[:,plotslice,:],cmap=plt.cm.gray)
ax[1].set_title('Side view')
ax[2].imshow(binary[:,:,plotslice],cmap=plt.cm.gray)
ax[2].set_title('Front view')

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Step 5: Segmentation and labelling
As seen above it is possible to separate the cell walls from the interior of the cells using a carefully selected intensity threshold. What we want to do now is to identify each individual cell in the sample. We'll do this using the [watershed](https://en.wikipedia.org/wiki/Watershed_(image_processing)) transformation. The first step is to compute a distance map from the binary image we created in the previous step. Each voxel in the distance transform will contain the distance (in voxels) to the closest 0 in the binary image. In the next step all local maximas with distances larger than 5 voxels are identified. These are used as seeds for the watershed function, i.e. they are the deepest part of the basins which are being filled during the watersheding. After watersheding we have identified and labelled each individual foam cell in the sample.

In [5]:
start,end =0,400 # Change these numbers to select a smaller part of the dataset
dmap = ndi.distance_transform_edt(binary) #distance transform
local_maxi = h_maxima(dmap[start:end],5)
markers = measure.label(local_maxi) #Use local maximas of the distance map as seeds for the watershed
labels = watershed(-dmap[start:end], markers, mask=binary[start:end]) #Watershed


In [6]:
mid = start + (end-start)//2 # for plotting

fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(binary[mid]*mask[mid], cmap=plt.cm.gray)
ax[0].set_title('Overlapping objects')
ax[1].imshow(-dmap[mid]*mask[mid], cmap=plt.cm.gray)
ax[1].set_title('Distances')
ax[2].imshow(labels[(end-start)//2]*mask[mid], cmap=plt.cm.nipy_spectral)
ax[2].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()

fig,axes = plt.subplots(1,3)
ax = axes.ravel()
print(labels.shape, (end-start)//2)
ax[0].imshow(labels[(end-start)//2,:,:]*mask[mid,:,:],cmap=plt.cm.nipy_spectral)
ax[0].set_title('Top view')
ax[1].imshow(labels[:,(end-start)//2,:]*mask[:,mid,:],cmap=plt.cm.nipy_spectral)
ax[1].set_title('Side view')
ax[2].imshow(labels[:,:,(end-start)//2]*mask[:,:,mid],cmap=plt.cm.nipy_spectral)
ax[2].set_title('Front view')

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(400, 507, 496) 200


## Step 6: Quantification
We are now ready to extract some numbers from the labelled dataset. For example we can look at the size distribution of the cells.

In [8]:
voxsize = 3
props = measure.regionprops(labels*mask, intensity_image=data) #This will compute a buch of properties of each label
max_sizes = [voxsize*p.major_axis_length for p in props]
min_sizes = [voxsize*p.minor_axis_length for p in props]
    
fig,axes = plt.subplots(1,2,sharey=True)
ax = axes.ravel()
ax[0].hist(min_sizes,bins=30,density=True,range=(0,300))
ax[0].set_ylabel('Probability density')
ax[0].set_xlabel('Minor axis length [micron]')
ax[1].hist(max_sizes,bins=30,density=True,range=(0,900))
ax[1].set_xlabel('Major axis length [micron]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …